In [46]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [47]:
df = pd.read_csv("raw_data.csv", index_col=0) 

In [48]:
df.head(3)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,51,No,NaN,2015.722222,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,"16280,83$","42330,17$",7,Y,No,13,"3,0",3,Full Time,0,NaN,5,"3,0",20,NaN,15,15,"16280,83$",1972,"195370,00$",NaN,NaN,Yes
1,52,No,NaN,2063.388889,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,"43331,17$",0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5,"3,0",33,NaN,11,9,NaN,1971,"199990,00$",NaN,NaN,1
2,42,No,travel_rarely,1984.253968,Research & Development,4,2,Technical Degree,1,3,3,0,NaN,3,5,ManaGER,4,Married,NaN,"41669,33$",1,NaN,No,11,"3,0",4,NaN,0,"22,0",3,NaN,22,NaN,11,15,NaN,1981,"192320,00$",ManaGER - Research & Development,NaN,1


**Transformamos: quitamos $ y cambiamos "," por "."**


`sameasmonthlyincome`, `salary`, `monthlyincome`, `monthlyrate`, `performancerating`, `worklifebalance`, `totalworkingyears`, `yearsincurrentrole`

In [49]:
# Lista de columnas a transformar
lista_col = [
    "salary", "monthlyincome", "monthlyrate",
    "performancerating", "worklifebalance", "totalworkingyears", 
]

# Lista para guardar errores
lista_errores = []

# Función para limpiar y convertir a float
def cambiar_comas(dato):
    if pd.isna(dato):
        return np.nan

    if not isinstance(dato, str):
        return float(dato)

    dato_limpio = dato.replace("$", "").replace(",", ".").strip()
    
    if dato_limpio == "":
        return np.nan

    try:
        return float(dato_limpio)
    except:
        lista_errores.append(dato)
        return np.nan




In [50]:
for col in lista_col:
    if col in df.columns:
        df[col] = df[col].apply(cambiar_comas)


In [51]:
df[["salary", "monthlyincome", "monthlyrate", "performancerating", "worklifebalance", "totalworkingyears"]]

,salary,monthlyincome,monthlyrate,performancerating,worklifebalance,totalworkingyears
0,195370.00,16280.83,42330.17,3.0,3.0,NaN
1,199990.00,NaN,43331.17,3.0,3.0,34.0
2,192320.00,NaN,41669.33,3.0,NaN,22.0
3,171690.00,14307.50,37199.50,3.0,NaN,NaN
4,NaN,12783.92,33238.20,3.0,3.0,NaN
...,...,...,...,...,...,...
1673,NaN,3949.17,10267.83,3.0,3.0,NaN
1674,191324.62,15943.72,41453.67,3.0,3.0,27.0
1675,28111.13,NaN,6090.75,3.0,3.0,6.0
1676,100071.84,8339.32,21682.23,NaN,3.0,NaN


**Transformamos: redondeamos a 2 decimales `hourlyrate`, `dailyrate`**

In [52]:
# Lista de columnas a redondear
columnas_a_redondear = ["hourlyrate", "dailyrate"]

# Función segura para redondear
def redondear_dos_decimales(valor):
    if pd.isna(valor):
        return np.nan  # conserva NaN o None sin error
    
    try:
        return round(float(valor), 2)
    except:
        return np.nan

# Aplicar a todas las columnas de la lista
for col in columnas_a_redondear:
    if col in df.columns:
        df[col] = df[col].apply(redondear_dos_decimales)


In [53]:
df[["dailyrate", "hourlyrate"]]

,dailyrate,hourlyrate
0,2015.72,NaN
1,2063.39,NaN
2,1984.25,NaN
3,1771.40,NaN
4,1582.77,NaN
...,...,...
1673,488.94,NaN
1674,1973.98,NaN
1675,290.04,NaN
1676,1032.49,NaN


**Calculamos y rellenamos las columnas ``salary``, ``hourlyrate`` y ``monthlyincome``**

In [54]:
# Rellenar hourlyrate donde esté vacío usando dailyrate / 8
df["hourlyrate"] = df["hourlyrate"].fillna(df["dailyrate"] / 8)


In [55]:
# Rellenar salary donde esté vacío usando monthlyincome * 12
df["salary"] = df["salary"].fillna(df["monthlyincome"] * 12)

In [56]:
# Rellenar monthlyincome donde esté vacío usando salary / 12
df["monthlyincome"] = df["monthlyincome"].fillna(df["salary"] / 12)

**Transformamos: modificamos columnas a minuscula**


`"department`, `educationfield`, `attrition`, `jobrole`, `maritalstatus`, `over18`, `overtime`, `standardhours`, `roledepartament`, `remotework`

In [57]:
cambios_a_minuscula = ["department", "educationfield", "attrition", "jobrole", "maritalstatus", "over18", "overtime", "standardhours", "roledepartament", "remotework"]

In [58]:
for col in cambios_a_minuscula:
    df[col] = df[col].apply(lambda dato: dato.lower() if type(dato) == str else dato)

In [59]:
df[["department", "educationfield", "attrition", "jobrole", "maritalstatus", "over18", "overtime", "standardhours", "roledepartament", "remotework"]]

,department,educationfield,attrition,jobrole,maritalstatus,over18,overtime,standardhours,roledepartament,remotework
0,NaN,NaN,no,research director,NaN,y,no,full time,NaN,yes
1,NaN,life sciences,no,manager,NaN,NaN,NaN,NaN,NaN,1
2,research & development,technical degree,no,manager,married,NaN,no,NaN,manager - research & development,1
3,NaN,medical,no,research director,married,y,NaN,full time,NaN,false
4,NaN,technical degree,no,sales executive,divorced,y,no,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
1673,NaN,medical,no,research scientist,single,NaN,NaN,NaN,NaN,yes
1674,NaN,NaN,no,manager,married,y,no,full time,NaN,false
1675,NaN,NaN,no,research scientist,NaN,NaN,no,part time,NaN,false
1676,NaN,life sciences,no,manufacturing director,divorced,NaN,yes,part time,NaN,yes


**Transformamos: espaciado sobrante en valores de la columna ``department``, ``jobrole`` y ``educationfield``**

In [60]:
print(df['department'].unique())
print('-------------')
print(df['jobrole'].unique())
print(df['educationfield'].unique())

[nan ' research & development ' ' sales ' ' human resources ']
-------------
[' research director ' ' manager ' ' sales executive '
 ' manufacturing director ' ' research scientist '
 ' healthcare representative ' ' laboratory technician '
 ' sales representative ' ' human resources ']
[nan 'life sciences' 'technical degree' 'medical' 'other' 'marketing'
 'human resources']


In [61]:
def sin_espaciado_extra(series):
    return series.str.strip().str.replace(r'\s+', ' ', regex=True)

columnas_a_limpiar = ['department', 'jobrole', 'educationfield']  # Ejemplo de columnas

for c in columnas_a_limpiar:

    if c in df.columns:  
        df[c] = sin_espaciado_extra(df[c])


In [62]:
print(df['department'].unique())
print('-------------')
print(df['jobrole'].unique())
print('-------------')
print(df['educationfield'].unique())

# no nos hace falta limpiar 'roledepartment' porque está destinada a eliminación

[nan 'research & development' 'sales' 'human resources']
-------------
['research director' 'manager' 'sales executive' 'manufacturing director'
 'research scientist' 'healthcare representative' 'laboratory technician'
 'sales representative' 'human resources']
-------------
[nan 'life sciences' 'technical degree' 'medical' 'other' 'marketing'
 'human resources']


**Transformamos: añadimos información a columna``department`` con información de ``jobrole``**

In [63]:
keywords = {
    "sales": ["sales"],
    "human resources": ["human"],
    "research & development": ["laboratory", "manufacturing", "research", "healthcare"]
}


In [64]:
def infer_department(jobrole, keywords_dict):
    if pd.isna(jobrole):
        return np.nan

    for dept, words in keywords_dict.items():
        for w in words:
            if w in jobrole:
                return dept

    return np.nan  # si no coincide con ninguna de las 3

In [65]:
df["department"] = df["jobrole"].apply(lambda x: infer_department(x, keywords))

In [66]:
df[["department", "jobrole", "roledepartament", "joblevel"]].sample(20)

,department,jobrole,roledepartament,joblevel
1542,NaN,manager,NaN,3
325,sales,sales executive,NaN,2
800,research & development,laboratory technician,laboratory technician - research & developm...,1
916,research & development,healthcare representative,NaN,2
1621,sales,sales representative,NaN,1
931,sales,sales representative,NaN,1
51,NaN,manager,NaN,4
1640,sales,sales executive,sales executive - sales,2
1632,research & development,laboratory technician,NaN,1
712,research & development,manufacturing director,NaN,2


**Transformamos: columna `maritalstatus` 1 dato mal escritos**

In [67]:
df["maritalstatus"].value_counts()

maritalstatus
married     419
single      343
divorced    205
marreid      36
Name: count, dtype: int64

In [68]:
df["maritalstatus"] = df["maritalstatus"].replace("marreid", "married")

In [69]:
df["maritalstatus"].value_counts()

maritalstatus
married     455
single      343
divorced    205
Name: count, dtype: int64

**REPLACE `maritalstatus`, `overtime`, `department` - NaN por "unknown"**

In [70]:
df = df.fillna({'maritalstatus': 'unknown', 'overtime': 'unknown', 'department': 'unknown'})

In [71]:
df[["maritalstatus", "overtime", "department"]].value_counts().reset_index()

,maritalstatus,overtime,department,count
0,unknown,no,research & development,187
1,unknown,unknown,research & development,165
2,married,no,research & development,123
3,married,unknown,research & development,120
4,single,no,research & development,89
5,unknown,unknown,sales,87
6,single,unknown,research & development,84
7,unknown,no,sales,65
8,married,unknown,sales,60
9,divorced,no,research & development,59


**Transformamos: valores NaN por 'non-travel' en columna ``businesstravel`` basados en la información que nos ha proporcionado nuestro enlace con el proyecto**

In [72]:
df["businesstravel"] = df["businesstravel"].fillna("non-travel")

In [73]:
df['businesstravel'].value_counts()

businesstravel
non-travel           894
travel_rarely        616
travel_frequently    168
Name: count, dtype: int64

**DROP Columns - `employeecount`, `sameasmonthlyincome`,`numberchildren`, `over18`, `yearsincurrentrole`**

In [74]:
df = df.drop(['employeecount', 'sameasmonthlyincome', 'numberchildren', 'over18', 'yearsincurrentrole','roledepartament'], axis=1)

In [75]:
df.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework
0,51,no,non-travel,2015.72,research & development,6,3,NaN,1,1,0,251.96500,3,5,research director,3,unknown,16280.830000,42330.17,7,no,13,3.0,3,full time,0,NaN,5,3.0,20,15,15,1972,195370.0,yes
1,52,no,non-travel,2063.39,unknown,1,4,life sciences,2,3,0,257.92375,2,5,manager,3,unknown,16665.833333,43331.17,0,unknown,14,3.0,1,NaN,1,34.0,5,3.0,33,11,9,1971,199990.0,1


**KEEP en BBDD pero no en CSV:**
- numberchildren (todos nan, pero puede ser interesante para la empresa en el futuro)

**DROP Duplicate Rows - `employeenumber` - keep first, reset index**

Duplicados borrados, quardamos solo el primero, reset indice del df.

In [76]:
df = df.drop_duplicates(subset='employeenumber', keep='first').reset_index(drop=True)

In [77]:
df['employeenumber'].duplicated().any()

np.False_

**REPLACE `genders` - 0 = m, 1 = f**

In [78]:
df['gender'] = df['gender'].replace({0: 'm', 1: 'f'})

**FILLNA `standardhours` - NaN = full time**

In [79]:
df['standardhours'] = df['standardhours'].fillna('full time')

**MODIFICANDO columna 'AGE'**

In [80]:
df['age'].unique()

array(['51', '52', '42', '47', '46', '48', '59', '41', '56', '38', '55',
       '40', '58', '35', '45', '33', '36', '34', 'forty-seven', '53',
       '43', '60', '32', '37', '49', '39', '50', '44', '30',
       'fifty-eight', '29', '31', '54', '57', '27', 'thirty-six', '28',
       '26', 'fifty-five', '25', 'fifty-two', 'thirty-one', '24',
       'thirty', '23', '22', '21', '20', 'twenty-six', '19',
       'thirty-seven', '18', 'thirty-two', 'twenty-four'], dtype=object)

In [81]:
np.sort(df['age'].unique())

array(['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39',
       '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50',
       '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
       'fifty-eight', 'fifty-five', 'fifty-two', 'forty-seven', 'thirty',
       'thirty-one', 'thirty-seven', 'thirty-six', 'thirty-two',
       'twenty-four', 'twenty-six'], dtype=object)

In [82]:
# valores = np.array(['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50','51', '52', '53', '54', '55', '56', '57', '58', '59', '60','fifty-eight', 'fifty-five', 'fifty-two', 'forty-seven', 'thirty', 'thirty-one', 'thirty-seven', 'thirty-six', 'thirty-two', 'twenty-four', 'twenty-six'], dtype=object)

In [83]:
dict_edades = {'fifty-eight':'58', 'fifty-five':'55', 'fifty-two':'52', 'forty-seven':'47', 'thirty':'30', 'thirty-one':'31', 'thirty-seven':'37', 'thirty-six':'36', 'thirty-two':'32', 'twenty-four':'34', 'twenty-six':'26'}

In [84]:
df['age'] = df['age'].replace(dict_edades)
df['age'] = df['age'].astype(int)

In [85]:
df['age'].unique()

array([51, 52, 42, 47, 46, 48, 59, 41, 56, 38, 55, 40, 58, 35, 45, 33, 36,
       34, 53, 43, 60, 32, 37, 49, 39, 50, 44, 30, 29, 31, 54, 57, 27, 28,
       26, 25, 24, 23, 22, 21, 20, 19, 18])

In [86]:
# Función para convertir cada edad
# def convertir_edad(edad):
#    return dict_edades.get(edad, edad)

# Aplicar la función a toda la columna 'age'
# df['age'] = df['age'].apply(convertir_edad)

# Guardar el DataFrame actualizado
# df.to_csv('tu_archivo_actualizado.csv', index=False)

**Transformando valores a absolutos**

In [87]:
df["distancefromhome"] = df["distancefromhome"].abs()

In [88]:
df['distancefromhome'].unique()

array([ 6,  1,  4,  2,  3, 22, 25,  9,  7, 23, 10, 12, 14, 13, 15,  8, 42,
       28, 37,  5, 16, 35, 26, 24, 29, 17, 21, 18, 30, 27, 20, 31, 39, 11,
       19, 33, 34, 46, 36, 45, 47, 32, 41, 49, 48, 38, 43, 40, 44])

**GUARDAR datos limpios a .csv nuevo "raw_data_limpio.csv"**

In [89]:
df.to_csv("raw_data_limpio_2.csv", index=False)

In [90]:
df.shape
# raw_data.csv original: 1678 rows, 41 columns
# raw_data_limpio_2.csv: 1614 rows, 35 columns

(1614, 35)